# Visualize suspected leukemia cells

#### This notebook labels each cell within this patient's images by it's Phenograph cluster number generated in Phenograph-CD45_features.ipynb. Suspected Lk cells are outlined in red to visually confirm.

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile
import cv2
from PIL import Image
from PIL import ImageDraw
from skimage import data, color, io, img_as_float
from os import listdir
from os.path import isfile, join
import skimage.io
import skimage.util
from skimage import data, segmentation, util, measure
from skimage.segmentation import expand_labels

In [22]:
# create dictionary of directories for each patient
img_dirs = {'P51':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P51_R1158_S1_Normalized_DAPI_B2M_CD34/', 
            'P52':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P52_R1149_S1_Normalized_DAPI_B2M_CD34/', 
            'P53':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P53_R1149_S2_Normalized_DAPI_B2M_CD34/', 
            'P56':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P56_R1158_S2_Normalized_DAPI_B2M_CD34/', 
            'P57':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P57_R1158_S3_Normalized_DAPI_B2M_CD34/', 
            'P58':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P58_R1149_S3_Normalized_DAPI_B2M_CD34/'}

In [23]:
patient = 'P58'

In [24]:
# create dictionary of directories for each patient
mask_dir = '/data/Zhaolab/1_AMLCosMx/Final_scripts/2_Segmentation/3_NucMemMerging/' + patient + '_hybrid/labels_predicted_2_15_24/'

In [25]:
# load in celltype predictions for all cells
morph_predicted = pd.read_csv(patient + '_CD34_subclusters.csv')
morph_predicted

,Unnamed: 0,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters
0,P58_FOV01_cell_5,1,5,A,2915.0,2191.777778,1.0,3,0
1,P58_FOV01_cell_6,1,6,A,1327.0,1207.444444,1.0,3,11
2,P58_FOV01_cell_40,1,40,A,2162.0,851.944444,1.0,3,7
3,P58_FOV01_cell_50,1,50,A,1312.0,1378.944444,1.0,3,3
4,P58_FOV01_cell_64,1,64,A,1877.0,2390.111111,1.0,3,1
...,...,...,...,...,...,...,...,...,...
5805,P58_FOV23_cell_7292,23,7292,C,1105.0,1072.444444,1.0,3,15
5806,P58_FOV23_cell_7312,23,7312,C,1729.0,971.666667,1.0,3,6
5807,P58_FOV23_cell_7357,23,7357,C,770.0,960.333333,1.0,3,7
5808,P58_FOV23_cell_7785,23,7785,C,1216.0,891.666667,1.0,3,16


In [6]:
morph_predicted = morph_predicted.drop(columns=["Unnamed: 0"])
morph_predicted

,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1.1,PhenoGraph_clusters
0,1,5,A,2915.0,2191.777778,1.0,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1,5
1,1,50,A,1312.0,1378.944444,1.0,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1,6
2,1,64,A,1877.0,2390.111111,1.0,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1,3
3,1,124,A,1726.0,9412.500000,1.0,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1,0
4,1,174,A,1413.0,1864.111111,1.0,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1,7
...,...,...,...,...,...,...,...,...,...
1899,23,4090,C,992.0,1125.944444,1.0,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1,8
1900,23,5074,C,940.0,1413.000000,1.0,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1,3
1901,23,6205,C,1363.0,1116.888889,1.0,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1,8
1902,23,6390,C,601.0,1865.000000,1.0,PhenoGraph_clusters_round_1,PhenoGraph_clusters_round_1,6


In [26]:
# add col to indicate if cell is predicted CD34+ lk cell
morph_predicted['Lk_cell'] = 0
morph_predicted['Lk_cell'][morph_predicted['PhenoGraph_clusters'].isin([0,1,3])] = 1
morph_predicted

/tmp/ipykernel_1595231/2483237797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morph_predicted['Lk_cell'][morph_predicted['PhenoGraph_clusters'].isin([0,1,3])] = 1


,Unnamed: 0,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters,Lk_cell
0,P58_FOV01_cell_5,1,5,A,2915.0,2191.777778,1.0,3,0,1
1,P58_FOV01_cell_6,1,6,A,1327.0,1207.444444,1.0,3,11,0
2,P58_FOV01_cell_40,1,40,A,2162.0,851.944444,1.0,3,7,0
3,P58_FOV01_cell_50,1,50,A,1312.0,1378.944444,1.0,3,3,1
4,P58_FOV01_cell_64,1,64,A,1877.0,2390.111111,1.0,3,1,1
...,...,...,...,...,...,...,...,...,...,...
5805,P58_FOV23_cell_7292,23,7292,C,1105.0,1072.444444,1.0,3,15,0
5806,P58_FOV23_cell_7312,23,7312,C,1729.0,971.666667,1.0,3,6,0
5807,P58_FOV23_cell_7357,23,7357,C,770.0,960.333333,1.0,3,7,0
5808,P58_FOV23_cell_7785,23,7785,C,1216.0,891.666667,1.0,3,16,0


In [27]:
morph_predicted[morph_predicted['Lk_cell'] == 1]

,Unnamed: 0,FOV,cell_ID,Timepoint,Size (pixels),CD34_median,Passed_QC,PhenoGraph_clusters_round_1,PhenoGraph_clusters,Lk_cell
0,P58_FOV01_cell_5,1,5,A,2915.0,2191.777778,1.0,3,0,1
3,P58_FOV01_cell_50,1,50,A,1312.0,1378.944444,1.0,3,3,1
4,P58_FOV01_cell_64,1,64,A,1877.0,2390.111111,1.0,3,1,1
13,P58_FOV01_cell_124,1,124,A,1726.0,9412.500000,1.0,3,0,1
18,P58_FOV01_cell_174,1,174,A,1413.0,1864.111111,1.0,3,1,1
...,...,...,...,...,...,...,...,...,...,...
5787,P58_FOV23_cell_4090,23,4090,C,992.0,1125.944444,1.0,3,3,1
5799,P58_FOV23_cell_5074,23,5074,C,940.0,1413.000000,1.0,3,1,1
5800,P58_FOV23_cell_6205,23,6205,C,1363.0,1116.888889,1.0,3,1,1
5801,P58_FOV23_cell_6390,23,6390,C,601.0,1865.000000,1.0,3,3,1


In [28]:
# save list of all patient's CD34+ cells
pos_to_save = morph_predicted[morph_predicted['Lk_cell'] == 1][['FOV', 'cell_ID']]
pos_to_save['Patient'] = patient

### Visualize all cells with cluster names, highlight likely Lk cells

In [29]:
# create list of fovs
fovs = list(set(morph_predicted['FOV'].tolist()))
fovs.sort()
    
for fov in fovs:
    
    # subset FOV        
    one_fov = morph_predicted[morph_predicted['FOV'] == fov]
    
    # Load img
    location = img_dirs[patient]
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            img = cv2.imread(location + item)
            print(item)
            break
            
    # brighten image
    # define the alpha and beta
    alpha = 1.5 # Contrast control
    beta = 35 # Brightness control

    # call convertScaleAbs function
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    # Load masks
    location = mask_dir
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break

    # get all mask values
    fov_cells = np.unique(labels)
    
    # Load centroids
    coords_dir = mask_dir + 'cell_centroids/'
    dirlist = listdir(coords_dir)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
            
    # merge prediction and centroid tables
    one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)
    one_fov = one_fov.rename(columns={"cell_ID": "label"})
                
    combo = pd.merge(one_fov,centroids,left_on='label',right_on='label',how='left')
    
    # visualize all segmented cells
    boundaries = segmentation.find_boundaries(labels, connectivity=1, mode='inner', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [255,255,0]
    
    # create list of CD34+ cells
    pos_cells = combo[combo['Lk_cell'] == 1]['label'].tolist()
    pos_cells = [int(i) for i in pos_cells]
    # set all labels of other clusters to zero
    labels_edited = labels.copy()
    for cell in fov_cells:
        if cell in pos_cells:
            pass
        else:
            labels_edited[labels_edited == cell] = 0

    # add colored outline
    boundaries = segmentation.find_boundaries(labels_edited, connectivity=1, mode='thick', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [0,0,255]

    # get unique phenograph clusters
    clusters = list(set(combo['PhenoGraph_clusters'].tolist()))
    
    for cluster in clusters:

        # create list of this cluster's cells
        pos_cells_df = combo[combo['PhenoGraph_clusters'] == cluster]

        for k in range(pos_cells_df.shape[0]):
            img = cv2.putText(img, str(cluster), (int(pos_cells_df['x'].iloc[k]), int(pos_cells_df['y'].iloc[k])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255,255,255], 2, cv2.LINE_AA)
            
    # save image
    cv2.imwrite('CD34_subclusters_overlaid/' + patient + '_FOV' + str(fov).zfill(2) + '_CD34_subcluster.png', img)

20220215_094003_S3_C902_P99_N99_F001_normalized.png
20220215_094003_S3_C902_P99_N99_F001_normalized_cp_masks.tif


/tmp/ipykernel_1595231/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S3_C902_P99_N99_F002_normalized.png
20220215_094003_S3_C902_P99_N99_F002_normalized_cp_masks.tif


/tmp/ipykernel_1595231/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S3_C902_P99_N99_F003_normalized.png
20220215_094003_S3_C902_P99_N99_F003_normalized_cp_masks.tif


...

/tmp/ipykernel_1595231/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S3_C902_P99_N99_F022_normalized.png
20220215_094003_S3_C902_P99_N99_F022_normalized_cp_masks.tif


/tmp/ipykernel_1595231/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S3_C902_P99_N99_F023_normalized.png
20220215_094003_S3_C902_P99_N99_F023_normalized_cp_masks.tif


/tmp/ipykernel_1595231/1622947058.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


## Before saving leukemia cell calls, remove few megakaryocytes classfied as leukemia cells

In [30]:
pos_to_save_copy = pos_to_save.copy()
pos_to_save_copy['New_ID'] = pos_to_save_copy['Patient'] + '_FOV' + pos_to_save_copy['FOV'].astype(str).str.zfill(2) + "_cell_" + pos_to_save_copy["cell_ID"].astype(str)
pos_to_save_copy = pos_to_save_copy.set_index('New_ID')
pos_to_save_copy

,FOV,cell_ID,Patient
New_ID,,,
P58_FOV01_cell_5,1,5,P58
P58_FOV01_cell_50,1,50,P58
P58_FOV01_cell_64,1,64,P58
P58_FOV01_cell_124,1,124,P58
P58_FOV01_cell_174,1,174,P58
...,...,...,...
P58_FOV23_cell_4090,23,4090,P58
P58_FOV23_cell_5074,23,5074,P58
P58_FOV23_cell_6205,23,6205,P58


In [31]:
# read in cell size data
cellData = pd.read_csv('/data/Zhaolab/1_AMLCosMx/Final_scripts/4_CellTyping/2_FilterbySize/Cells_filtbyRBCandsize.csv', index_col=0)
cellData = cellData[cellData['Patient'] == patient]
cellData

,label,area,y,x,Patient,FOV,RBC
ID,,,,,,,
P58_FOV01_cell_1,1,931,15.024705,2640.207304,P58,1,0
P58_FOV01_cell_2,2,1597,22.125861,374.536631,P58,1,0
P58_FOV01_cell_3,3,1788,23.072148,1331.709172,P58,1,0
P58_FOV01_cell_4,4,3501,29.445587,1694.469009,P58,1,0
P58_FOV01_cell_5,5,2915,26.315266,1893.304631,P58,1,0
...,...,...,...,...,...,...,...
P58_FOV23_cell_7966,7966,731,3638.218878,4327.127223,P58,23,0
P58_FOV23_cell_7971,7971,596,3640.771812,4117.850671,P58,23,0
P58_FOV23_cell_7972,7972,359,3641.122563,4992.493036,P58,23,0


In [32]:
pos_to_save_copy = pos_to_save_copy.merge(cellData['area'], how='left', right_index=True, left_index=True)
pos_to_save_copy.sort_values(by='area', ascending=False)

,FOV,cell_ID,Patient,area
New_ID,,,,
P58_FOV09_cell_597,9,597,P58,29170
P58_FOV01_cell_375,1,375,P58,25015
P58_FOV06_cell_2848,6,2848,P58,10537
P58_FOV07_cell_2216,7,2216,P58,7060
P58_FOV04_cell_2350,4,2350,P58,5775
...,...,...,...,...
P58_FOV01_cell_6530,1,6530,P58,371
P58_FOV01_cell_7009,1,7009,P58,345
P58_FOV06_cell_11835,6,11835,P58,335


In [35]:
# remove incorrectly called megakaryocytes
pos_to_save_copy = pos_to_save_copy.drop(axis=0, index=['P58_FOV09_cell_597', 'P58_FOV01_cell_375', 'P58_FOV06_cell_2848'])
pos_to_save_copy

,FOV,cell_ID,Patient,area
New_ID,,,,
P58_FOV01_cell_5,1,5,P58,2915
P58_FOV01_cell_50,1,50,P58,1312
P58_FOV01_cell_64,1,64,P58,1877
P58_FOV01_cell_124,1,124,P58,1726
P58_FOV01_cell_174,1,174,P58,1413
...,...,...,...,...
P58_FOV23_cell_4090,23,4090,P58,992
P58_FOV23_cell_5074,23,5074,P58,940
P58_FOV23_cell_6205,23,6205,P58,1363


In [36]:
# save list of CD34+ lk cells
pos_to_save_copy[['FOV', 'cell_ID', 'Patient']].to_csv(patient + '_LkCells.csv', index=False)